## MLP sobre dados de Sintomas da Lombalgia

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix

Para esse projeto, usaremos redes MLPs em cima do conjunto de dados de Sintomas de Lombalgia com o objetivo de identificar pessoas que podem ter a doença a partir de determinadas caracteristicas. Começaremos importando o dataset e dando uma olhada nos seus dados:

In [ ]:
df = pd.read_csv('/kaggle/input/lower-back-pain-symptoms-dataset/Dataset_spine.csv')
df.head()

Nesse conjunto, vemos a presença de uma coluna com o nome "Unnamed: 13" que não nos passa nenhum tipo de informação. Por isso, vamos retirá-la do dataset para que não nos atrapalhe

In [ ]:
df = df.drop(['Unnamed: 13'], axis=1)
df.head()

Com esse conjunto de dados, vamos separar o X (atributos) e o y (labels) e dividi-lo em conjunto de treino e de teste com uma proporção de 80/20, ou seja, 80% dos dados disponíveis serão usados no treino do MLP e os outros 20% para testar o modelo

In [ ]:
y = df['Class_att']
X = df.drop(['Class_att'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 13)

# Rede MLP com parâmetros padrões

Agora, vamos criar o modelo usando a biblioteca SKLearn. Para um primeiro teste, usaremos os parâmetros que estão como padrão na implementação que são dados abaixo:

In [ ]:
clf_default = MLPClassifier(random_state = 13, verbose = True)
clf_default.get_params()

Com tais parâmetros, vamos treinar o modelo usando o nosso X_test e y_test

In [ ]:
clf_default.fit(X_train, y_train)

In [ ]:
y_pred = clf_default.predict(X_test)
clf_default.score(X_test, y_test)

In [ ]:
class_report = classification_report(y_test, y_pred)
print(class_report)

Com esses parâmetros, obtemos uma acurácia de 83%. Além disso, podemos tirar mais alguns dados do classification report que calculamos:

1. **Precisão (porcentagem dos classificados como X que eram realmente X)**   
Aqui, tivemos 92% para *abnormal* e 71% para *normal*, ou seja, vemos que o nosso modelo classificou muitas pessoas como *normal* e que na verdade essas pessoas eram *abnormal*. A chance de uma pessoa ser classificada erroneamente como saudável, quando na verdade é doente, é de 29%.

2. **Recall (quando realmente é da classe X, o quão frequente você classifica como X)**   
Obtemos 83% para *abnormal* e 85% para *normal*. Dessa forma, 15% das pessoas saudáveis foram erroneamente classificadas como doentes. Essa taxa aumenta para 17% no caso dos doentes.


Dando uma olhada nos nossos dados novamente, vemos que existe quase o dobro de amostras referentes a *abnormal* em comparação com *normal*. Isso pode ser uma explicação do porquê uma maior porcentagem de pessoas saudáveis está sendo classificada erroneamente como doentes.

In [ ]:
y.value_counts()

# Rede MLP com parâmetros alterados

Para melhorar a performance dos experimentos vamos tentar compreender a relevância de cada coluna para o classificador e remover as colunas que tenham valores que possam atrapalhar o processo de aprendizado e classificação, para isso vamos tentar buscar por características que nos apontem problemas com uma dada informação como sua variância por grupo

In [ ]:
mask = df['Class_att'] == 'Abnormal'
abnormal_df = df[mask]
normal_df = df[~mask]

In [ ]:
abnormal_df.head()

In [ ]:
normal_df.head()

Agora vamos computar a variância as colunas de cada subconjunto

In [ ]:
normal_df.var()

In [ ]:
abnormal_df.var()

Podemos notar algumas colunas muito próximas e muito discrepantes, podemos encontrar nelas informações interessantes para continuar nossa análise, vamos partir desse ponto...

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns

sns.set()
sns.set_theme()

In [ ]:
for key in normal_df:
    if key == 'Class_att':
        continue
    print(key)

In [ ]:
for key in df:
    if key == 'Class_att':
        continue
    sns.displot(data=df,x=key, hue='Class_att', kde=True)

Podemos visualizar graficamente assim as variáveis que não falam muito sobre o problema, o que também parece falar sobre uma tendência de aumento de valores para algumas variáveis, essas variáveis que podem causar grande mudança são as que devemos focar e tentar combiná-las de forma que seja possível construir uma melhor distribuição de dados, que torne mais clara a diferença, construindo um parâmetro ou um conjunto combinado de parâmetros que melhore o resultado, mudar as metodologias também para visualizar quais parâmetros que tem maior influência nos dados. 

Dessa forma vamos focar exclusivamente nas colunas de 1 a 6.

In [ ]:
import numpy as np

def f(x):
    return np.exp(1 + (x['Col5'] - x['Col6'])/max(x['Col5'], x['Col6']))

df1 = df.assign(Col13 = df[['Col5', 'Col6']].apply(f, axis=1))
df1.head()

In [ ]:
sns.displot(data=df1,x='Col13', hue='Class_att', kde=True)
sns.displot(data=df1,x='Col6', hue='Class_att', kde=True)

In [ ]:
modified_df = df1[['Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6', 'Col13']]

Agora vamos montar o classificador com dados diferentes para encontrar o maior valor possível.

In [ ]:
clf_mod = MLPClassifier(hidden_layer_sizes=(100,100,100), 
                            random_state = 21, 
                            activation='relu', 
                            verbose = True,
                           early_stopping=True,
                           validation_fraction=0.2)
clf_mod.get_params()

In [ ]:
X_mod = modified_df
Y_mod = df['Class_att']
X_train_mod, X_valid_mod, y_train_mod, y_valid_mod = train_test_split(X, y, test_size = 0.1, random_state = 21)

In [ ]:
clf_mod.fit(X_train_mod, y_train_mod)

In [ ]:
y_pred = clf_mod.predict(X_valid_mod)
clf_mod.score(X_valid_mod, y_valid_mod)

In [ ]:
class_report = classification_report(y_valid_mod, y_pred)
print(class_report)